In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install openai

In [5]:
import os
from datetime import datetime, timezone
from google.cloud import storage
from dotenv import load_dotenv
load_dotenv("/Users/paigegiese/SYG/landproDATA_code/misc-work/.env")
from openai import OpenAI
import logging
import json
import time
import random
import pandas as pd
from typing import TypedDict, List, Dict, Any, Optional
from pydantic import BaseModel, Field
from typing import TypedDict, List, Dict, Any, Optional, Annotated, Literal
oai_client = OpenAI(api_key =os.getenv("LANDPRO_OAI_KEY"))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/paigegiese/SYG/landproDATA_admin/landprodata-server-65e4697bdfe1.json'


In [55]:
blobbies = []
for blob in recent_blobs:
    #if 'MOVADO' in blob.name.upper():
    print(blob.name, blob.time_created)
    blobbies.append([blob.name.split(' - ')[0],blob.time_created,blob.name])

Subdivisions/10887 - ID - Payette - EVANS-DONOHO ADDITION.01.pdf 2025-01-21 16:24:41.514000+00:00
Subdivisions/10887 - ID - Payette - EVANS-DONOHO ADDITION.01.pdf.txt 2025-01-21 16:24:41.932000+00:00
Subdivisions/10991 - ID - Payette - GREEN HILL TRACT.01.pdf 2025-01-21 16:32:26.634000+00:00
Subdivisions/10991 - ID - Payette - GREEN HILL TRACT.01.pdf.txt 2025-01-21 16:32:27.019000+00:00
Subdivisions/11 - ID - Ada - EASTON.01.pdf.txt 2025-04-08 01:02:22.198000+00:00
Subdivisions/14757 - ID - Boise - LAZY J RANCH EST.01.pdf 2025-01-17 19:03:11.167000+00:00
Subdivisions/14757 - ID - Boise - LAZY J RANCH EST.01.pdf.txt 2025-01-18 23:06:09.112000+00:00
Subdivisions/14865 - ID - Boise - STEPHENSON'S SUBDIVISION.01.pdf 2025-01-17 19:03:39.478000+00:00
Subdivisions/14865 - ID - Boise - STEPHENSON'S SUBDIVISION.01.pdf.txt 2025-01-18 23:05:48.114000+00:00
Subdivisions/15548 - ID - Clearwater - CLEARWATER PLATEAU SUBDIVISION.01.pdf 2025-01-21 15:28:35.429000+00:00
Subdivisions/15548 - ID - Clearw

In [58]:
blob_df = pd.DataFrame(blobbies, columns=['blob_name_short','time_created','blob_name'])
(blob_df['blob_name_short'].value_counts() ==2).sum()

np.int64(518)

In [60]:
blob_df['file_ct'] = blob_df.groupby('blob_name_short')['blob_name'].transform('count')

In [70]:
test_blobs = blob_df.loc[blob_df['file_ct']==2]['blob_name_short'].drop_duplicates().sample(150).tolist()

test_blob_df = blob_df.loc[blob_df['blob_name_short'].isin(test_blobs)].sort_values(by=['blob_name_short','blob_name'], ascending=True).reset_index(drop=True)
test_blob_df.to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/ocr/test_documents.csv', index=False)

In [243]:
class Person(BaseModel):
    first_name: Annotated[str, Field(description="A person's first name")]
    last_name: Annotated[str, Field(description="A person's last name")]
    title: Annotated[str, Field(description="A person's job title")]
    license_number: Annotated[str, Field(description="A person's license number, if applicable (likely for plats and surveys), where the person is a certified professional")]

class Entities(BaseModel):
    name: Annotated[str, Field(description="The entity's full name (including suffices like 'LLC', or 'inc')")]
    type: Annotated[Literal['state', 'county', 'city','company','parcel_name','other'], Field(description="Identify entities in the document and categorize by type. Use 'other' if the existing types don't fit the context")]
    inferred_context: Annotated[str, Field(description="A 200-character description of why this entity is present. Why are they referenced? What appears to be the purpose of their presence?")]

class TownshipSectionRange(BaseModel):
    range: Annotated[str, Field(description="Range")]
    section: Annotated[str, Field(description="Section")]
    township: Annotated[str, Field(description="Township")]

class Document(BaseModel):
    people: Annotated[Optional[List[Person]], Field("Returns a list of people identified from the extracted text")] 
    entities: Annotated[Optional[List[Entities]], Field("Returns a list of non-human entities identified from the extracted text")] 
    township_section_range: Annotated[TownshipSectionRange, Field(description="Township, section, and range. Return None if not present.")]
    legal_description: Annotated[str, Field(description="Legal description from the extracted text - it is critical this is extremely precise to the original text.")]
    #confidence_in_ocr: Annotated[float, Field(description="Based on the text, how confident are you that the OCR process was of acceptable quality? 0-100 scale only.")]
    #confidence_in_interpretation: Annotated[float, Field(description="Based on the text, how confident are you that you've interpreted the text appropriately? 0-100 scale only.")]

def ocr(oai_client, text):
    response = oai_client.responses.parse(
        model="gpt-4.1-mini",
        input=[
            {"role": "system", "content": "You are a helpful assistant at interpreting raw text extracted from complex land and parcel documents and surveys. You will always be provided one document at a time. \
             ##Precise instructions:\
             1. Carefully analyze the extracted text from the OCR process. Each input will delineate the files with the words '[FILE NAME]', which indicates page separation within the same document. \
                Similarly, the OCR processor identifies different paragraphs, which are delineated with [PARAGRAPH]. This should help make logical assumptions about the request. \
                Do not use prior knowledge or information from outside the context to answer the questions. Only use the information provided in the context to answer the questions.\
             2. Review the required JSON structure for the response\
             3. Fulfill the request to the best of your ability\
             4.  Review the input text structure to ensure the Legal Description is as complete and precise as possible. \
             Hints to help find the legal description: \
             - Use paragraph and header/subtitle hints, like isolating text between headers.\
             - Legal descriptions often explain a geographic polygon that is able to be closed when drawn.\
             - Note language that describes an area of ownerhip\
             - Note language like 'the point of beginning', and subsequent measurements. e.g. South 00°38 16 West, 305.80 feet\
             - Note when a block/paragraph of text seemingly terminates with an acreage statement.\
            ### A complete example of a legal description \
            # [PARAGRAPH] # \
            A portion of Lots 25, 26, and 27, Roberts and Hill Subdivision as is filed in Book 4 of Plats at Page 159, records of Ada County, Idaho located in the\
            Southeast 1/4 of the Northeast 1/4 of Section 14, T.4N., R.1E., B.M., City of Boise, Ada County, Idaho more particularly described as follows:\
            # [PARAGRAPH] # \
            Commencing at the East 1/4 corner of said Section 14, from which the Center 1/4 corner of said Section 14 bears North 88°50 47 West, 2635.27 feet;\
            thence on the East boundary line of said Section 14, North 00°38 16 East, 1185.68 feet; thence leaving said East boundary line, North 88°35 49 West, 25.00\
            feet to the westerly right-of-way line of N. Bogart Lane and the REAL POINT OF BEGINNING;\
            # [PARAGRAPH] # \
            thence on said westerly right-of-way line the following seven (7) courses and distances:\
            # [PARAGRAPH] # \
            South 00°38 16  West, 305.80 feet;\
            South 07°14 08  West, 65.87 feet;\
            # [PARAGRAPH] # \
            South 03°53 20  West, 48.39 feet;\
            # [PARAGRAPH] # \
            71.67 feet along the arc of curve to the right having a radius of 60.00 feet, a central angle of 68°26 22  and a long chord which bears South\
            38°06 30  West, 67.48 feet;\
            # [PARAGRAPH] # \
            52.27 feet along the arc of curve to the right having a radius of 135.00 feet, a central angle of 22°11 07  and a long chord which bears South\
            83°25 15  West, 51.95 feet;\
            # [PARAGRAPH] # \
            81.83 feet along the arc of a curve to the left having a radius of 615.00 feet, a central angle of 07°37 26  and a long chord which bears North\
            89°17 54  West, 81.77 feet;\
            # [PARAGRAPH] #  \
            16.80 feet along the arc of a curve to the right having a radius of 585.00 feet, a central angle of 01°38 44  and a long chord which bears South\
            87°42 44  West, 16.80 feet to the northerly right-of-way line of W. Hill Road Parkway;\
            # [PARAGRAPH] # \
            thence on said northerly right-of-way line the following two (2) courses and distances:\
            # [PARAGRAPH] # \
            103.13 feet along the arc of a curve to the left having a radius of 1,193.92 feet, a central angle of 04°56 57  and a long chord which bears North\
            86°30 32  West, 103.10 feet;\
            # [PARAGRAPH] # \
            North 88°59 01  West, 659.42 feet to the west boundary line of said Lot 27, Roberts and Hill Subdivision;\
            # [PARAGRAPH] # \
            thence on said west boundary line, North 00°35 32  East, 608.79 feet to the north boundary line of said Roberts and Hill Subdivision;\
            thence on said north boundary line, South 88°35 49  East, 814.40 feet;\
            # [PARAGRAPH] # \
            thence leaving said north boundary line, South 00°38 16  West, 125.00 feet;\
            thence South 88°35 49  East, 150.00 feet to the REAL POINT OF BEGINNING.\
            # [PARAGRAPH] # \
            Containing 12.93 acres, more or less.\
            ## Formatting Instructions\
             Use as precise of language as possible. Do not include any explanation in the reply. Only include the extracted information in the reply.\
             Only in 'inferred_context' are you allowed to practice freedom of explanation."},
            {"role": "user", "content": f"Return your interpretation of the following OCR text using the structured output model provided: {text}"}
        ],
        text_format = Document
    )
    return response


In [150]:
bb = get_blob(bucket, blob_name = "Subdivisions/15036 - ID - Ada - MOVADO SUBDIVISION NO 1.01.pdf.txt").download_as_text()

In [151]:
print(bb)

BOOK 112, PAGE_110331,
N 1/4 CORNER
CP&F # 108008840
BASIS OF BEARINGS
S 89°13'35" E 2676.87"
E. OVERLAND RD.
SEC CORNER
CP&F # 105155109
16 | 15
-- ----
21 | 22
PLAT SHOWING
MOVADO SUBDIVISION NO. 1
LOCATED IN THE NE1/4 OF SECTION 21,
T. 3 N., R. 1 E., B.M., CITY OF MERIDIAN, ADA COUNTY, IDAHO
2017
PLAT NOTES
1. EASEMENTS: A PERMANENT EASEMENT FOR PUBLIC UTILITIES AND PROPERTY DRAINAGE
IS HEREBY RESERVED ALONG THE FOLLOWING:
-TEN (10') FOOT WIDE ADJACENT TO SUBDIVISION BOUNDARY
-TEN (10') FOOT WIDE ADJACENT TO PUBLIC RIGHT OF WAYS
-THREE (3') FOOT WIDE ADJACENT TO EITHER SIDE OF INTERIOR LOT LINES EXCEPT FOR
WHERE ZERO LOT LINES ARE SHOWN
SEE DETAIL A
S 64040'54"
-A PERMANENT EASEMENT FOR NAMPA/MERIDIAN IRRIGATION DISTRICT IS HEREBY
RESERVED ALONG THE FOLLOWING:
-TEN (10') FOOT WIDE ALONG SUBDIVISION BOUNDARY, REAR LOT LINES AND AS DEPICTED
48.32
N
UNPLATTED |
1260.44'
DETAIL A
S. CLOVERDALE RD.
L
50.17'
N 7°28'47" W
47.59X
R
_
SCALE 1" = 30'_
-
A PERMANENT EASEMENT FOR PRESSURE IRRIG

In [152]:
response = ocr(bb)


In [153]:
response.model_dump()

{'id': 'resp_6845acb48988819c810e870cb7f71f320e24b66b9ba05bd7',
 'created_at': 1749396660.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-mini-2025-04-14',
 'object': 'response',
 'output': [{'id': 'msg_6845acb5373c819c87703860c714c6360e24b66b9ba05bd7',
   'content': [{'annotations': [],
     'text': '{"people":[{"first_name":"Jim D.","last_name":"Conger","title":"Member","license_number":""},{"first_name":"Carl","last_name":"Porter","title":"Professional Land Surveyor","license_number":"PLS 14221"},{"first_name":"Jerry L.","last_name":"Hasting","title":"Professional Land Surveyor","license_number":"PLS 5359"},{"first_name":"Christopher D.","last_name":"Rich","title":"City Engineer","license_number":""}],"entities":[{"name":"City of Meridian","type":"city","inferred_context":"City authority approving and servicing the subdivision including water and sanitation."},{"name":"Ada County","type":"county","inferred_context":"County g

In [155]:
response.output_parsed.model_dump()

{'people': [{'first_name': 'Jim D.',
   'last_name': 'Conger',
   'title': 'Member',
   'license_number': ''},
  {'first_name': 'Carl',
   'last_name': 'Porter',
   'title': 'Professional Land Surveyor',
   'license_number': 'PLS 14221'},
  {'first_name': 'Jerry L.',
   'last_name': 'Hasting',
   'title': 'Professional Land Surveyor',
   'license_number': 'PLS 5359'},
  {'first_name': 'Christopher D.',
   'last_name': 'Rich',
   'title': 'City Engineer',
   'license_number': ''}],
 'entities': [{'name': 'City of Meridian',
   'type': 'city',
   'inferred_context': 'City authority approving and servicing the subdivision including water and sanitation.'},
  {'name': 'Ada County',
   'type': 'county',
   'inferred_context': 'County governing jurisdiction where the subdivision is located with surveying and tax certification responsibilities.'},
  {'name': 'Movado Development, LLC',
   'type': 'company',
   'inferred_context': 'Owner and developer of the subdivision.'},
  {'name': 'Movado C

## Running comparisons

Unstract API

In [180]:
import requests
load_dotenv("/Users/paigegiese/SYG/landproDATA_code/misc-work/.env", override=True)
unstract_key =os.getenv("UNSTRACT_API")

def run_unstract(file_name):
    print('fetching pdf bytes')
    bb = get_blob(bucket, blob_name = file_name).download_as_bytes() #MUST BE .PDF
    print(bb[:50])
    api_url = 'https://us-central.unstract.com/deployment/api/org_bffz77gWEUOQuVrX/unstract_ocr/'
    headers = {
    'Authorization': f'Bearer {unstract_key}'
    }
    payload = {'timeout': 300, 'include_metadata': False}
    # filepath = '/path/to/file'
    files=[('files',('file',bb,'application/octet-stream'))]
    print('requesting unstract api')
    response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
    resp=response.json()
    res = resp['message']['result'][0]['result']['output']
    return resp, res

def run_openai(file_name):
    print('fetching text blob')
    bb = get_blob(bucket, blob_name = file_name).download_as_text() #MUST BE .TXT
    print(bb[:50])
    print('calling openai')
    response = ocr(bb)
    res = response.output_parsed.model_dump()
    response = response.model_dump()

    return response, res



In [131]:
test_blob_df.sort_values(by=['time_created','blob_name'], ascending=[False,False], inplace=True)
test_blob_df.reset_index(drop=True, inplace=True)
test_blob_df

,blob_name_short,time_created,blob_name,file_ct
0,Subdivisions/66819,2025-06-05 17:56:40.442000+00:00,Subdivisions/66819 - ID - Bonneville - 1785304...,2
1,Subdivisions/66819,2025-06-05 17:56:40.074000+00:00,Subdivisions/66819 - ID - Bonneville - 1785304...,2
2,Subdivisions/66816,2025-06-05 17:54:53.054000+00:00,Subdivisions/66816 - ID - Bonneville - 1784675...,2
3,Subdivisions/66816,2025-06-05 17:54:52.525000+00:00,Subdivisions/66816 - ID - Bonneville - 1784675...,2
4,Subdivisions/66809,2025-06-05 17:50:28.755000+00:00,Subdivisions/66809 - ID - Bonneville - 1783123...,2
...,...,...,...,...
295,Subdivisions/66369,2025-01-08 17:03:07.957000+00:00,Subdivisions/66369 - ID - Ada - JUMP CREEK SUB...,2
296,Subdivisions/66344,2025-01-04 23:09:12.533000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
297,Subdivisions/66344,2025-01-03 19:01:53.324000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
298,Subdivisions/66327,2025-01-02 15:45:30.709000+00:00,Subdivisions/66327 - ID - Bannock - WESTFIELD ...,2


In [186]:
test_blob_df

,blob_name_short,time_created,blob_name,file_ct
0,Subdivisions/66819,2025-06-05 17:56:40.442000+00:00,Subdivisions/66819 - ID - Bonneville - 1785304...,2
1,Subdivisions/66819,2025-06-05 17:56:40.074000+00:00,Subdivisions/66819 - ID - Bonneville - 1785304...,2
2,Subdivisions/66816,2025-06-05 17:54:53.054000+00:00,Subdivisions/66816 - ID - Bonneville - 1784675...,2
3,Subdivisions/66816,2025-06-05 17:54:52.525000+00:00,Subdivisions/66816 - ID - Bonneville - 1784675...,2
4,Subdivisions/66809,2025-06-05 17:50:28.755000+00:00,Subdivisions/66809 - ID - Bonneville - 1783123...,2
...,...,...,...,...
295,Subdivisions/66369,2025-01-08 17:03:07.957000+00:00,Subdivisions/66369 - ID - Ada - JUMP CREEK SUB...,2
296,Subdivisions/66344,2025-01-04 23:09:12.533000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
297,Subdivisions/66344,2025-01-03 19:01:53.324000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
298,Subdivisions/66327,2025-01-02 15:45:30.709000+00:00,Subdivisions/66327 - ID - Bannock - WESTFIELD ...,2


In [196]:
responses_challenge = []
test_blob_df2 = test_blob_df.iloc[13:,:]
test_blob_df2.reset_index(drop=True, inplace=True)

for i, j in test_blob_df2.iterrows():
    if j['blob_name'].endswith('.pdf'):
        try:
            print(f'Running unstract on {j["blob_name"]}. Index: {i}')
            response, answer = run_unstract(file_name= j['blob_name'])
        except Exception as e:
            print(f"⚠️ Failed to process {j['blob_name']}: {e}")
    else:
        try:
            response, answer = run_openai(j['blob_name'])
            print(f'Running openai on {j["blob_name"]}. Index: {i}')
        except Exception as e:
            print(f"⚠️ Failed to process {j['blob_name']}: {e}")
    
    responses_challenge.append([j['blob_name_short'],j['blob_name'], response, answer])



fetching text blob
⚠️ Failed to process Subdivisions/66783 - ID - Fremont - TETON HOMESTEAD DIVISION NO 1 - 1ST AMENDED PLAT.01.PDF: 'utf-8' codec can't decode byte 0x9c in position 647: invalid start byte
fetching text blob
BOUNDARY DESCRIPTION:
SITUATED IN THE STATE OF IDA
calling openai
Running openai on Subdivisions/66786 - ID - Fremont - JOHNSON HOMESTEAD 1ST AMENDED PLAT.01.PDF.txt. Index: 1
fetching text blob
⚠️ Failed to process Subdivisions/66786 - ID - Fremont - JOHNSON HOMESTEAD 1ST AMENDED PLAT.01.PDF: 'utf-8' codec can't decode byte 0x9c in position 647: invalid start byte
fetching text blob
NS81°43'24"W 558,82"
LOT 1
1.00 AC.
N86°08'09"E 55
calling openai
Running openai on Subdivisions/66787 - ID - Fremont - BETTY'S HOMESITE DIVISION NO 1.01.PDF.txt. Index: 3
fetching text blob
⚠️ Failed to process Subdivisions/66787 - ID - Fremont - BETTY'S HOMESITE DIVISION NO 1.01.PDF: 'utf-8' codec can't decode byte 0x9c in position 647: invalid start byte
fetching text blob
LEGEND
40

In [182]:
#response, answer = run_unstract('Subdivisions/15036 - ID - Ada - MOVADO SUBDIVISION NO 1.01.pdf')

In [197]:
test_output = pd.DataFrame(responses_challenge, columns=['blob_name_short','blob_name','response','answer'])
test_output

,blob_name_short,blob_name,response,answer
0,Subdivisions/66783,Subdivisions/66783 - ID - Fremont - TETON HOME...,{'id': 'resp_6845b335a54c819dbb48fb0359fcfeea0...,"{'people': [{'first_name': 'Destin', 'last_nam..."
1,Subdivisions/66786,Subdivisions/66786 - ID - Fremont - JOHNSON HO...,{'id': 'resp_6845b6a4a97081a2a11385b7f7c3a7180...,"{'people': [{'first_name': 'Todd', 'last_name'..."
2,Subdivisions/66786,Subdivisions/66786 - ID - Fremont - JOHNSON HO...,{'id': 'resp_6845b6a4a97081a2a11385b7f7c3a7180...,"{'people': [{'first_name': 'Todd', 'last_name'..."
3,Subdivisions/66787,Subdivisions/66787 - ID - Fremont - BETTY'S HO...,{'id': 'resp_6845b6af8e0c8192935748a4c8e68bca0...,"{'people': [{'first_name': 'Betty R.', 'last_n..."
4,Subdivisions/66787,Subdivisions/66787 - ID - Fremont - BETTY'S HO...,{'id': 'resp_6845b6af8e0c8192935748a4c8e68bca0...,"{'people': [{'first_name': 'Betty R.', 'last_n..."
...,...,...,...,...
282,Subdivisions/66369,Subdivisions/66369 - ID - Ada - JUMP CREEK SUB...,{'id': 'resp_6845c78f9754819286302dfde0310eba0...,"{'people': [{'first_name': 'Cody M.', 'last_na..."
283,Subdivisions/66344,Subdivisions/66344 - ID - Canyon - MASON CREEK...,{'id': 'resp_6845c7ab4ac881a080a08d0abf05eea80...,"{'people': [{'first_name': 'Michael', 'last_na..."
284,Subdivisions/66344,Subdivisions/66344 - ID - Canyon - MASON CREEK...,{'id': 'resp_6845c7ab4ac881a080a08d0abf05eea80...,"{'people': [{'first_name': 'Michael', 'last_na..."
285,Subdivisions/66327,Subdivisions/66327 - ID - Bannock - WESTFIELD ...,{'id': 'resp_6845c7c93c3481929b681a6188f18ae30...,"{'people': [{'first_name': 'DANNIS', 'last_nam..."


In [ ]:
final_df = test_output.copy()

In [200]:
final_final = pd.concat([final_df, test_output])
final_final.reset_index(drop=True, inplace=True)

final_final['ct']= final_final.groupby('blob_name_short')['blob_name'].transform('count')

final_final.loc[final_final['ct']==2]['blob_name_short'].nunique()

150

In [207]:
final_final.loc[final_final['ct']==2].to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/ocr/test_documents_successful_responses.csv', index=False)#.pivot(index='blob_name_short',columns=['blob_name'])

In [211]:
final_final2 = final_final.loc[final_final['ct']==2]

f_texts = final_final2.loc[final_final2['blob_name'].str.endswith('.txt')]
f_texts.rename(columns={'answer':'text_answer'}, inplace=True)
f_pdfs = final_final2.loc[final_final2['blob_name'].str.endswith('.pdf')]
f_pdfs.rename(columns={'answer':'ocr_answer'}, inplace=True)

final_wide = f_texts[['blob_name_short','text_answer']].merge(f_pdfs[['blob_name_short','blob_name','ocr_answer']], on='blob_name_short')



/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_25631/3920552873.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_texts.rename(columns={'answer':'text_answer'}, inplace=True)
/var/folders/9v/6bsrztp11777s06bs8q25y2h0000gn/T/ipykernel_25631/3920552873.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_pdfs.rename(columns={'answer':'ocr_answer'}, inplace=True)


In [194]:
test_blob_df.iloc[13:,:]

,blob_name_short,time_created,blob_name,file_ct
13,Subdivisions/66783,2025-05-23 17:38:58.625000+00:00,Subdivisions/66783 - ID - Fremont - TETON HOME...,2
14,Subdivisions/66786,2025-05-23 17:36:21.766000+00:00,Subdivisions/66786 - ID - Fremont - JOHNSON HO...,2
15,Subdivisions/66786,2025-05-23 17:36:21.402000+00:00,Subdivisions/66786 - ID - Fremont - JOHNSON HO...,2
16,Subdivisions/66787,2025-05-23 17:35:43.524000+00:00,Subdivisions/66787 - ID - Fremont - BETTY'S HO...,2
17,Subdivisions/66787,2025-05-23 17:35:43.130000+00:00,Subdivisions/66787 - ID - Fremont - BETTY'S HO...,2
...,...,...,...,...
295,Subdivisions/66369,2025-01-08 17:03:07.957000+00:00,Subdivisions/66369 - ID - Ada - JUMP CREEK SUB...,2
296,Subdivisions/66344,2025-01-04 23:09:12.533000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
297,Subdivisions/66344,2025-01-03 19:01:53.324000+00:00,Subdivisions/66344 - ID - Canyon - MASON CREEK...,2
298,Subdivisions/66327,2025-01-02 15:45:30.709000+00:00,Subdivisions/66327 - ID - Bannock - WESTFIELD ...,2


In [214]:
final_wide['text_answer'].iloc[0]

{'people': [{'first_name': 'Spencer',
   'last_name': 'McCutcheon',
   'title': 'Licensed Professional Land Surveyor',
   'license_number': '20907'},
  {'first_name': 'Chris',
   'last_name': 'Canfield',
   'title': 'City Engineer',
   'license_number': '10551'},
  {'first_name': 'Kenneth Baldwin',
   'last_name': 'Roberts',
   'title': 'City Surveyor',
   'license_number': '9755'},
  {'first_name': 'David',
   'last_name': 'Romrell',
   'title': 'Bonneville County Surveyor',
   'license_number': '12223'},
  {'first_name': 'Christopher',
   'last_name': 'Carney',
   'title': 'Manager of Carney Real Estate Enterprise LLC',
   'license_number': ''},
  {'first_name': 'Austin',
   'last_name': 'Ray',
   'title': 'Notary Public',
   'license_number': '20221026'}],
 'entities': [{'name': 'Bonneville County',
   'type': 'county',
   'inferred_context': 'Location of the recorded subdivision plat.'},
  {'name': 'City of Idaho Falls',
   'type': 'city',
   'inferred_context': 'Governing municipa

In [248]:
ocr_adj = []

for i, j in final_wide.iterrows():
    try:
        temp= j['ocr_answer']['people']
        temp.update(j['ocr_answer']['entities'])
        temp.update(j['ocr_answer']['township_section_range'])
        temp.update({'legal_description':j['ocr_answer']['legal_description']})
        ocr_adj.append(temp)
    except:
        print(i)
        ocr_adj.append(j['ocr_answer'])
    


10
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


In [253]:
final_wide.iloc[143]['text_answer']==final_wide.iloc[143]['ocr_answer']

True

In [255]:
final_wide['text_answer'].iloc[0]

{'people': [{'first_name': 'Spencer',
   'last_name': 'McCutcheon',
   'title': 'Licensed Professional Land Surveyor',
   'license_number': '20907'},
  {'first_name': 'Chris',
   'last_name': 'Canfield',
   'title': 'City Engineer',
   'license_number': '10551'},
  {'first_name': 'Kenneth Baldwin',
   'last_name': 'Roberts',
   'title': 'City Surveyor',
   'license_number': '9755'},
  {'first_name': 'David',
   'last_name': 'Romrell',
   'title': 'Bonneville County Surveyor',
   'license_number': '12223'},
  {'first_name': 'Christopher',
   'last_name': 'Carney',
   'title': 'Manager of Carney Real Estate Enterprise LLC',
   'license_number': ''},
  {'first_name': 'Austin',
   'last_name': 'Ray',
   'title': 'Notary Public',
   'license_number': '20221026'}],
 'entities': [{'name': 'Bonneville County',
   'type': 'county',
   'inferred_context': 'Location of the recorded subdivision plat.'},
  {'name': 'City of Idaho Falls',
   'type': 'city',
   'inferred_context': 'Governing municipa

In [257]:
final_wide['ocr_answer_sorted'] = ocr_adj

final_wide.to_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/ocr/test_documents_successful_responses_wide.csv', index=False)

In [280]:
dd = pd.read_csv('/Users/paigegiese/SYG/landproDATA_code/misc-work/ocr/test_documents_successful_responses_wide.csv')[['blob_name_short','blob_name','text_answer','ocr_answer_sorted']].to_dict('records')

In [281]:
def sort_json_people_entities(data):
    # Sort people by full name (first + last)
    if 'people' in data:
        data['people'] = sorted(
            data['people'],
            key=lambda x: f"{x.get('first_name', '')} {x.get('last_name', '')}"
        )
    # Sort entities by 'name'
    if 'entities' in data:
        data['entities'] = sorted(
            data['entities'],
            key=lambda x: x.get('name', '')
        )
    return data

In [284]:
import ast
for r in dd:
    #print(r)
    try:
        r['ocr_answer_sorted'] = sort_json_people_entities(ast.literal_eval(r['ocr_answer_sorted']))
        r['text_answer']= sort_json_people_entities(ast.literal_eval(r['text_answer']))
    except: 
        continue

In [285]:
dd[0]

{'blob_name_short': 'Subdivisions/66819',
 'blob_name': 'Subdivisions/66819 - ID - Bonneville - 1785304.01.pdf',
 'text_answer': {'people': [{'first_name': 'Austin',
    'last_name': 'Ray',
    'title': 'Notary Public',
    'license_number': '20221026'},
   {'first_name': 'Chris',
    'last_name': 'Canfield',
    'title': 'City Engineer',
    'license_number': '10551'},
   {'first_name': 'Christopher',
    'last_name': 'Carney',
    'title': 'Manager of Carney Real Estate Enterprise LLC',
    'license_number': ''},
   {'first_name': 'David',
    'last_name': 'Romrell',
    'title': 'Bonneville County Surveyor',
    'license_number': '12223'},
   {'first_name': 'Kenneth Baldwin',
    'last_name': 'Roberts',
    'title': 'City Surveyor',
    'license_number': '9755'},
   {'first_name': 'Spencer',
    'last_name': 'McCutcheon',
    'title': 'Licensed Professional Land Surveyor',
    'license_number': '20907'}],
  'entities': [{'name': 'Austad Construction',
    'type': 'company',
    'inf

## Processing from the prepared_files file path

In [6]:
client = storage.Client()
bucket = client.get_bucket("prepared-files")
blobs = bucket.list_blobs(prefix="2/12/001001/020100002/prepared")
cutoff = datetime(2025, 1, 1, tzinfo=timezone.utc)
#recent_blobs = [blob for blob in blobs if blob.time_created and blob.time_created > cutoff and not blob.name.endswith('.json')]
json_blobs = [blob for blob in blobs if blob.name.endswith('.json')]

def get_blob(bucket, blob_name):
    blob = bucket.blob(blob_name)
    return blob


In [107]:
blob_files = [json_blobs[name].name.split('prepared/')[1].split('_ocr')[0] for name in range(len(json_blobs))]

In [114]:
import numpy as np
np.max([int(blob_files[n].replace('S','')) for n in range(len(blob_files))])

2134503

### Get file groups together... 

In [ ]:
from collections import defaultdict
blob_files = [json_blobs[name].name.split('prepared/')[1].split('_ocr')[0] for name in range(len(json_blobs))]
def group_paths_by_prefix(paths, prefix_length=6):
    groups = defaultdict(list)
    for path in paths:
        prefix = path[:prefix_length]
        groups[prefix].append(path)
    return dict(groups)

In [129]:
groups = group_paths_by_prefix(blob_files)

for group, files in groups.items():
    print(f"Group {group}: {files}")

Group S20005: ['S2000501', 'S2000502', 'S2000503', 'S2000504']
Group S20009: ['S2000901', 'S2000902', 'S2000903', 'S2000904', 'S2000905']
Group S20014: ['S2001401', 'S2001402', 'S2001403', 'S2001404', 'S2001405']
Group S20019: ['S2001901', 'S2001902', 'S2001903', 'S2001904', 'S2001905', 'S2001906', 'S2001907']
Group S20026: ['S2002601', 'S2002602', 'S2002603', 'S2002604']
Group S20030: ['S2003001', 'S2003002', 'S2003003']
Group S20033: ['S2003301', 'S2003302', 'S2003303', 'S2003304', 'S2003305', 'S2003306']
Group S20039: ['S2003901', 'S2003902', 'S2003903', 'S2003904', 'S2003905', 'S2003906', 'S2003907', 'S2003908', 'S2003909']
Group S20048: ['S2004801', 'S2004802', 'S2004803', 'S2004804']
Group S20052: ['S2005201', 'S2005202', 'S2005203']
Group S20055: ['S2005501', 'S2005502', 'S2005503', 'S2005504', 'S2005505', 'S2005506', 'S2005507', 'S2005508']
Group S20063: ['S2006301', 'S2006302', 'S2006303', 'S2006304']
Group S20067: ['S2006701', 'S2006702', 'S2006703']
Group S20070: ['S2007001'

## Compile the formatted text for making the API call

In [210]:
def fetch_file_group(file_prefix):
    return ['2/12/001001/020100002/prepared/'+groups[file_prefix][fp]+'_ocr.json' for fp in range(len(groups[file_prefix]))]

def make_text(file_group):
    file_text = """"""

    for file in file_group:
        get_text = get_blob(bucket, file).download_as_string()
        get_json = json.loads(get_text)
        text = get_json['text']
        
        file_text += f'\n# [FILE NAME: {file}]# \n'
        
        for page in get_json['pages']:
            for para in (page['paragraphs']):
                #segments:
                segments = para['layout']['textAnchor']['textSegments']
                try:
                    start = int(segments[0]['startIndex'])
                except:
                    start = 0
                
                end = int(segments[0]['endIndex'])
                

                file_text += f"\n# [PARAGRAPH] # \n {text[start:end]}"
    
    return file_text


In [ ]:
groups.items()

dict_items([('S20005', ['S2000501', 'S2000502', 'S2000503', 'S2000504']), ('S20009', ['S2000901', 'S2000902', 'S2000903', 'S2000904', 'S2000905']), ('S20014', ['S2001401', 'S2001402', 'S2001403', 'S2001404', 'S2001405']), ('S20019', ['S2001901', 'S2001902', 'S2001903', 'S2001904', 'S2001905', 'S2001906', 'S2001907']), ('S20026', ['S2002601', 'S2002602', 'S2002603', 'S2002604']), ('S20030', ['S2003001', 'S2003002', 'S2003003']), ('S20033', ['S2003301', 'S2003302', 'S2003303', 'S2003304', 'S2003305', 'S2003306']), ('S20039', ['S2003901', 'S2003902', 'S2003903', 'S2003904', 'S2003905', 'S2003906', 'S2003907', 'S2003908', 'S2003909']), ('S20048', ['S2004801', 'S2004802', 'S2004803', 'S2004804']), ('S20052', ['S2005201', 'S2005202', 'S2005203']), ('S20055', ['S2005501', 'S2005502', 'S2005503', 'S2005504', 'S2005505', 'S2005506', 'S2005507', 'S2005508']), ('S20063', ['S2006301', 'S2006302', 'S2006303', 'S2006304']), ('S20067', ['S2006701', 'S2006702', 'S2006703']), ('S20070', ['S2007001', 'S

In [211]:
from itertools import islice
for group, files in islice(groups.items(),3):
    print(group)
    print(len(files))

S20005
4
S20009
5
S20014
5


In [212]:
files

['S2001401', 'S2001402', 'S2001403', 'S2001404', 'S2001405']

In [244]:
full_texts = []
for group, files in islice(groups.items(),6):
    print(f'Working on {group}')
    files_to_process = fetch_file_group(group)
    full_formatted_text = make_text(file_group=files_to_process)
    print(f'Appending full text for group {group}')
    full_texts.append([group,full_formatted_text])


Working on S20005
Appending full text for group S20005
Working on S20009
Appending full text for group S20009
Working on S20014
Appending full text for group S20014
Working on S20019
Appending full text for group S20019
Working on S20026
Appending full text for group S20026
Working on S20030
Appending full text for group S20030


In [250]:
ft = pd.DataFrame(full_texts, columns=['filename','full_text'])
print(ft['full_text'].iloc[-2])


# [FILE NAME: 2/12/001001/020100002/prepared/S2002601_ocr.json]# 

# [PARAGRAPH] # 
 W. MOON VALLEY RD.

# [PARAGRAPH] # 
 BALD

# [PARAGRAPH] # 
 S

# [PARAGRAPH] # 
 1. PACE

# [PARAGRAPH] # 
 STONEHART AVE.

# [PARAGRAPH] # 
 1994-

# [PARAGRAPH] # 
 144.07'

# [PARAGRAPH] # 
 N10°49'37"W

# [PARAGRAPH] # 
 BALD EAGLE POINTE SUBDIVISION NO. 2

# [PARAGRAPH] # 
 LOCATED IN THE S 1/2 OF SECTION 11 & THE N 1/2 OF SECTION 14
TOWNSHIP 4 NORTH, RANGE 1 WEST, B.M., EAGLE, ADA COUNTY, IDAHO
2023

# [PARAGRAPH] # 
 CP&F NO.
2014-058913

# [PARAGRAPH] # 
 1/4

# [PARAGRAPH] # 
 11 12

# [PARAGRAPH] # 
 100

# [PARAGRAPH] # 
 0

# [PARAGRAPH] # 
 100

# [PARAGRAPH] # 
 200

# [PARAGRAPH] # 
 300

# [PARAGRAPH] # 
 Scale: 1100'

# [PARAGRAPH] # 
 POINT OF BEGINNING

# [PARAGRAPH] # 
 W. STATE STREET
RMO

# [PARAGRAPH] # 
 HWY. 44

# [PARAGRAPH] # 
 BALD EAGLE POINTE SUBD. NO.

# [PARAGRAPH] # 
 79.01'
N80°15'15"E

# [PARAGRAPH] # 
 158.18'
N80°29'13"E

# [PARAGRAPH] # 
 2022-082123

# [PARAGRA

In [ ]:
tt = get_blob(bucket, '2/12/001001/020100002/prepared/S2000904_ocr_processed.txt').download_as_text()
#ld_old = ocr(oai_client = oai_client, text = tt)
print(ld_old.output_parsed.model_dump()['people'])

In [251]:
test_response = ocr(oai_client=oai_client, text = ft['full_text'].iloc[-2])

In [252]:
print(test_response.output_parsed.model_dump()['legal_description'])

A parcel being located in the S 1/2 of Section 11 and the N 1/2 of Section 14, Township 4 North, Range 1 West, Boise Meridian, City of Eagle, Ada County, Idaho, more particularly described as follows: Commencing at an aluminum cap monument marking the Southeast corner of the S 1/2 of said Section 11, from which an aluminum cap monument marking the Northeast corner of said S 1/2 bears N 0°49'08" E a distance of 2645.32 feet; Thence N 89°27'29" W along the southerly boundary of said S 1/2 a distance of 1842.61 feet to the point of beginning; Thence leaving said boundary S 0°46'32" W a distance of 476.06 feet to a point on the natural or ordinary high water line of the North Channel of the Boise River; Thence along said high water line the following described courses and distances: Thence S 63°22'38" W a distance of 150.89 feet to a point; Thence S 54°06'38" W a distance of 125.48 feet to a point; Thence S 53°06'49" W a distance of 11.60 feet to the Southeasterly corner of Bald Eagle Poin

Little audit of blocks vs paragraphs - paragraphs separates a little more liberally, but it all looks good. Just nuanced differences.

In [216]:
# blocks =[]
# # for each page
# for p in test['pages']:
#     # for each block
#     for block in (p['blocks']):
#         #print(block)
#         #segments:
#         segments = block['layout']['textAnchor']['textSegments']
#         try:
#             start = int(segments[0]['startIndex'])
#         except:
#             start = 0

#         blocks.append((start,int(segments[0]['endIndex'])))
        


    

In [217]:
# paragraphs =[]
# # for each page
# for p in test['pages']:
#     # for each paragraph
#     for para in (p['paragraphs']):
#         print(para)
#         #segments:
#         segments = para['layout']['textAnchor']['textSegments']
#         try:
#             start = int(segments[0]['startIndex'])
#         except:
#             start = 0

#         paragraphs.append((start,int(segments[0]['endIndex'])))
        


    

In [218]:
# for b in paragraphs[:20]:
#     print(text[b[0]:b[1]])
#     print('NEXT PARAGRAPH')

In [219]:
# for b in blocks:
#     print(text[b[0]:b[1]])
#     print('NEXT BLOCK')

In [220]:
# misses = []
# for p in paragraphs:
#     if p not in blocks:
#         misses.append(['in_paragraph',p])

# for b in blocks:
#     if b not in paragraphs:
#         misses.append(['in_blocks',b])


In [221]:
# for m in misses:
#     print(m)

In [222]:
# for m in misses:
#     if m[0] == 'in_paragraph':
#         print('In para, not blocks:\n')
#         print(text[m[1][0]:m[1][1]])
#     else:
#         print('In blocks, not para: \n')
#         print(text[m[1][0]:m[1][1]])
#     print('NEXT')


## Pseudo-code for Anthony to make the API call

In [ ]:
#####################
####### NOTES #######
#####################

# Files on the path: prepared-files/2/12/... [I think] are generated using the latest Document AI processor. 
# /prepared-files/2/12/001001/020100002/prepared contains Subdivisions. Subdivisions are n pages, depending on how many pages the original document has. Each PAGE has an accompanying .json and .txt file (i.e. 3 files per page).
# /prepared-files/2/12/001001/020200002/prepared contains ROS. ROS are 1 page, 1 PDF. Each PDF has an accompanying .json and .txt file (i.e. 3 files per document)
# TO MY KNOWLEDGE: Everything you will see IN PLATFORM currently, is sourced from landprodata-files/Subdivisions and were processed using the OLD vision model (not Document AI).

# Reference the accompanying screenshot for the modal David wants this service to run from. 

# STEP 1:
# For the Document in the .pdf file (visible) -> Send to new Document AI processer (OR pull it from the existing file paths (prepared_files/) noted above?? Not sure
# how he wants to reference this. Get the .json file(s). NOTE: Something is going on with the subdivisions files where they are producing a .json file for every page in the pdf. For this reason, see the recombine-files code below.
# 
# STEP 2:
# Recombine the files as necessary. 
# The following is the python code to fetch and recombine the prepared files. 

# Get the files -- AGAIN: MAY LOOK ENTIRELY DIFFERENT IN THE MODAL WORKFLOW
from collections import defaultdict
client = storage.Client()
bucket = client.get_bucket("prepared-files")
blobs = bucket.list_blobs(prefix="2/12/001001/020100002/prepared")
cutoff = datetime(2025, 1, 1, tzinfo=timezone.utc)
#recent_blobs = [blob for blob in blobs if blob.time_created and blob.time_created > cutoff and not blob.name.endswith('.json')]
json_blobs = [blob for blob in blobs if blob.name.endswith('.json')]

def get_blob(bucket, blob_name):
    blob = bucket.blob(blob_name)
    return blob

blob_files = [json_blobs[name].name.split('prepared/')[1].split('_ocr')[0] for name in range(len(json_blobs))]

def group_paths_by_prefix(paths, prefix_length=6):
    groups = defaultdict(list)
    for path in paths:
        prefix = path[:prefix_length]
        groups[prefix].append(path)
    return dict(groups)

groups = group_paths_by_prefix(blob_files)

def fetch_file_group(file_prefix):
    '''Only applicable when you're fetching files from the prepared-files bucket. May be entirely irrelevant if you are pulling from the landprodata-files bucket, and/or have the object
    represented differently in the modal workflow.'''
    return ['2/12/001001/020100002/prepared/'+groups[file_prefix][fp]+'_ocr.json' for fp in range(len(groups[file_prefix]))] 

## Step 2.5: This function is important because it formats the text from the json file to preserve the 'paragraph' structure, which improves the accuracy of the OCR interpretation.
def make_text(file_group):
    file_text = """"""

    for file in file_group:
        get_text = get_blob(bucket, file).download_as_string()
        get_json = json.loads(get_text)
        text = get_json['text']
        
        file_text += f'\n# [FILE NAME: {file}]# \n' ## KEY FORMATTING CODE - THESE LINES DENOTE PAGE SEPARATION IN THE TEXT OUTPUT
        
        for page in get_json['pages']:
            for para in (page['paragraphs']): ## KEY FORMATTING CODE - USE PARAGRAPHS KEY, NOT 'BLOCKS'
                #segments:
                segments = para['layout']['textAnchor']['textSegments']
                try:
                    start = int(segments[0]['startIndex'])
                except:
                    start = 0
                
                end = int(segments[0]['endIndex'])
                

                file_text += f"\n# [PARAGRAPH] # \n {text[start:end]}" ## KEY FORMATTING CODE - THESE LINES DENOTE PARAGRAPH SEPARATION IN THE TEXT OUTPUT
    
    return file_text
## STEP 3: Make the API call (see below, commented out PHP code equivalent)
# Using pydantic to define the structure of the expected output from the OCR process.
class Person(BaseModel):
    first_name: Annotated[str, Field(description="A person's first name")]
    last_name: Annotated[str, Field(description="A person's last name")]
    title: Annotated[str, Field(description="A person's job title")]
    license_number: Annotated[str, Field(description="A person's license number, if applicable (likely for plats and surveys), where the person is a certified professional")]

class Entities(BaseModel):
    name: Annotated[str, Field(description="The entity's full name (including suffices like 'LLC', or 'inc')")]
    type: Annotated[Literal['state', 'county', 'city','company','parcel_name','other'], Field(description="Identify entities in the document and categorize by type. Use 'other' if the existing types don't fit the context")]
    inferred_context: Annotated[str, Field(description="A 200-character description of why this entity is present. Why are they referenced? What appears to be the purpose of their presence?")]

class TownshipSectionRange(BaseModel):
    range: Annotated[str, Field(description="Range")]
    section: Annotated[str, Field(description="Section")]
    township: Annotated[str, Field(description="Township")]

class Document(BaseModel):
    people: Annotated[Optional[List[Person]], Field("Returns a list of people identified from the extracted text")] 
    entities: Annotated[Optional[List[Entities]], Field("Returns a list of non-human entities identified from the extracted text")] 
    township_section_range: Annotated[TownshipSectionRange, Field(description="Township, section, and range. Return None if not present.")]
    legal_description: Annotated[str, Field(description="Legal description from the extracted text - it is critical this is extremely precise to the original text.")]
    #confidence_in_ocr: Annotated[float, Field(description="Based on the text, how confident are you that the OCR process was of acceptable quality? 0-100 scale only.")]
    #confidence_in_interpretation: Annotated[float, Field(description="Based on the text, how confident are you that you've interpreted the text appropriately? 0-100 scale only.")]

def ocr(oai_client, text):
    response = oai_client.responses.parse(
        model="gpt-4.1-mini", ## THIS MODEL IS CHEAPER AND SUFFICIENT FOR THE TASK.
        input=[
            {"role": "system", "content": "You are a helpful assistant at interpreting raw text extracted from complex land and parcel documents and surveys. You will always be provided one document at a time. \
             ##Precise instructions:\
             1. Carefully analyze the extracted text from the OCR process. Each input will delineate the files with the words '[FILE NAME]', which indicates page separation within the same document. \
                Similarly, the OCR processor identifies different paragraphs, which are delineated with [PARAGRAPH]. This should help make logical assumptions about the request. \
                Do not use prior knowledge or information from outside the context to answer the questions. Only use the information provided in the context to answer the questions.\
             2. Review the required JSON structure for the response\
             3. Fulfill the request to the best of your ability\
             4.  Review the input text structure to ensure the Legal Description is as complete and precise as possible. \
             Hints to help find the legal description: \
             - Use paragraph and header/subtitle hints, like isolating text between headers.\
             - Legal descriptions often explain a geographic polygon that is able to be closed when drawn.\
             - Note language that describes an area of ownerhip\
             - Note language like 'the point of beginning', and subsequent measurements. e.g. South 00°38 16 West, 305.80 feet\
             - Note when a block/paragraph of text seemingly terminates with an acreage statement.\
            ### A complete example of a legal description \
            # [PARAGRAPH] # \
            A portion of Lots 25, 26, and 27, Roberts and Hill Subdivision as is filed in Book 4 of Plats at Page 159, records of Ada County, Idaho located in the\
            Southeast 1/4 of the Northeast 1/4 of Section 14, T.4N., R.1E., B.M., City of Boise, Ada County, Idaho more particularly described as follows:\
            # [PARAGRAPH] # \
            Commencing at the East 1/4 corner of said Section 14, from which the Center 1/4 corner of said Section 14 bears North 88°50 47 West, 2635.27 feet;\
            thence on the East boundary line of said Section 14, North 00°38 16 East, 1185.68 feet; thence leaving said East boundary line, North 88°35 49 West, 25.00\
            feet to the westerly right-of-way line of N. Bogart Lane and the REAL POINT OF BEGINNING;\
            # [PARAGRAPH] # \
            thence on said westerly right-of-way line the following seven (7) courses and distances:\
            # [PARAGRAPH] # \
            South 00°38 16  West, 305.80 feet;\
            South 07°14 08  West, 65.87 feet;\
            # [PARAGRAPH] # \
            South 03°53 20  West, 48.39 feet;\
            # [PARAGRAPH] # \
            71.67 feet along the arc of curve to the right having a radius of 60.00 feet, a central angle of 68°26 22  and a long chord which bears South\
            38°06 30  West, 67.48 feet;\
            # [PARAGRAPH] # \
            52.27 feet along the arc of curve to the right having a radius of 135.00 feet, a central angle of 22°11 07  and a long chord which bears South\
            83°25 15  West, 51.95 feet;\
            # [PARAGRAPH] # \
            81.83 feet along the arc of a curve to the left having a radius of 615.00 feet, a central angle of 07°37 26  and a long chord which bears North\
            89°17 54  West, 81.77 feet;\
            # [PARAGRAPH] #  \
            16.80 feet along the arc of a curve to the right having a radius of 585.00 feet, a central angle of 01°38 44  and a long chord which bears South\
            87°42 44  West, 16.80 feet to the northerly right-of-way line of W. Hill Road Parkway;\
            # [PARAGRAPH] # \
            thence on said northerly right-of-way line the following two (2) courses and distances:\
            # [PARAGRAPH] # \
            103.13 feet along the arc of a curve to the left having a radius of 1,193.92 feet, a central angle of 04°56 57  and a long chord which bears North\
            86°30 32  West, 103.10 feet;\
            # [PARAGRAPH] # \
            North 88°59 01  West, 659.42 feet to the west boundary line of said Lot 27, Roberts and Hill Subdivision;\
            # [PARAGRAPH] # \
            thence on said west boundary line, North 00°35 32  East, 608.79 feet to the north boundary line of said Roberts and Hill Subdivision;\
            thence on said north boundary line, South 88°35 49  East, 814.40 feet;\
            # [PARAGRAPH] # \
            thence leaving said north boundary line, South 00°38 16  West, 125.00 feet;\
            thence South 88°35 49  East, 150.00 feet to the REAL POINT OF BEGINNING.\
            # [PARAGRAPH] # \
            Containing 12.93 acres, more or less.\
            ## Formatting Instructions\
             Use as precise of language as possible. Do not include any explanation in the reply. Only include the extracted information in the reply.\
             Only in 'inferred_context' are you allowed to practice freedom of explanation."},
            {"role": "user", "content": f"Return your interpretation of the following OCR text using the structured output model provided: {text}"}
        ],
        text_format = Document
    )
    return response

## PHP VERSION OF THE ABOVE CODE




In [ ]:
# <?php

# class Person {
#     /** @var string A person's first name */
#     public string $first_name;

#     /** @var string A person's last name */
#     public string $last_name;

#     /** @var string A person's job title */
#     public string $title;

#     /** @var string A person's license number, if applicable */
#     public string $license_number;
# }

# class Entity {
#     /** @var string The entity's full name (including 'LLC', 'Inc', etc) */
#     public string $name;

#     /**
#      * @var string One of: 'state', 'county', 'city', 'company', 'parcel_name', 'other'
#      */
#     public string $type;

#     /** @var string Description of why this entity is referenced in the document */
#     public string $inferred_context;
# }

# class TownshipSectionRange {
#     /** @var string */
#     public string $range;

#     /** @var string */
#     public string $section;

#     /** @var string */
#     public string $township;
# }

# class Document {
#     /** @var Person[]|null */
#     public ?array $people = null;

#     /** @var Entity[]|null */
#     public ?array $entities = null;

#     /** @var TownshipSectionRange|null */
#     public ?TownshipSectionRange $township_section_range = null;

#     /** @var string Precise legal description from OCR text */
#     public string $legal_description;
# }


In [ ]:
## STEP 4: store the full response in a database log file

## STEP 5: Store the response.output_parsed.model_dump() in the OCR Matches column as a json object that we can later parse